# LSTM/Recurrent neural networks

*   Input data shape (12906, 658) with integer encoding.
*   The training accuracy with 100 epochs can lead to 97%. The following training accuracy with 50 epochs is up to 96.82%.
*   Training speed is the lowest among all seven methods.



In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
## adjust the length of DNA sequence with 658
## remove the uncorrect characters 'K','M', 'N', 'R', 'S', 'W', 'Y'
df = pd.read_csv (r'/content/train_features.csv')
def equal_length(seq):
    seq2 = seq.ljust(1058, '-') 
    fir = seq2[0:658]
    remove_characters = ['K','M', 'N', 'R', 'S', 'W', 'Y']
    for character in remove_characters:
        fir = fir.replace(character, "-")
    return fir
dna = df.dna.apply(equal_length)
## Example for arbitery DNA sequence
dna[3]

'AACATTATATTTTATTTTTGGTGCATGAGCTGGAATAGTAGGAACTTCATTAAGTATGCTAATTCGAGCTGAATTAGGAAACCCAGGTGCATTAATTGGAGATGACCAAATTTATAATGTTATTGTTACTGCACATGCATTTATTATGATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATATTAGGAGCCCCAGATATAGCTTTTCCTCGAATAAATAATATAAGTTTTTGATTACTTCCTCCTTCATTAACGCTCCTCTTAATAAGAAGTCTAGTG---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'

In [3]:
## Transfer Character to Int
from numpy import argmax
def num_encoder(seq):
  alphabet = 'ACGT-'
  char_to_int = dict((c, i) for i, c in enumerate(alphabet))
  to_array = [char_to_int[char] for char in seq]
  return(np.array(to_array).reshape((658)))

dna_code = np.array(dna.apply(num_encoder).to_list())
## Example
dna_code[3]

array([0, 0, 1, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 2, 3,
       2, 1, 0, 3, 2, 0, 2, 1, 3, 2, 2, 0, 0, 3, 0, 2, 3, 0, 2, 2, 0, 0,
       1, 3, 3, 1, 0, 3, 3, 0, 0, 2, 3, 0, 3, 2, 1, 3, 0, 0, 3, 3, 1, 2,
       0, 2, 1, 3, 2, 0, 0, 3, 3, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0, 2, 2, 3,
       2, 1, 0, 3, 3, 0, 0, 3, 3, 2, 2, 0, 2, 0, 3, 2, 0, 1, 1, 0, 0, 0,
       3, 3, 3, 0, 3, 0, 0, 3, 2, 3, 3, 0, 3, 3, 2, 3, 3, 0, 1, 3, 2, 1,
       0, 1, 0, 3, 2, 1, 0, 3, 3, 3, 0, 3, 3, 0, 3, 2, 0, 3, 3, 3, 3, 3,
       3, 3, 3, 0, 3, 0, 2, 3, 3, 0, 3, 0, 1, 1, 3, 0, 3, 3, 0, 3, 0, 0,
       3, 3, 2, 2, 0, 2, 2, 0, 3, 3, 3, 2, 2, 0, 0, 0, 3, 3, 2, 0, 3, 3,
       0, 2, 3, 3, 1, 1, 3, 3, 3, 0, 0, 3, 0, 3, 3, 0, 2, 2, 0, 2, 1, 1,
       1, 1, 0, 2, 0, 3, 0, 3, 0, 2, 1, 3, 3, 3, 3, 1, 1, 3, 1, 2, 0, 0,
       3, 0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 0, 2, 3, 3, 3, 3, 3, 2, 0, 3, 3,
       0, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 1, 0, 3, 3, 0, 0, 1, 2, 1, 3, 1,
       1, 3, 1, 3, 3, 0, 0, 3, 0, 0, 2, 0, 0, 2, 3,

In [11]:
y_train = pd.read_csv (r'/content/train_labels.csv')
ytrain = np.array(y_train['labels'], dtype = "uint32")


In [12]:
model = tf.keras.models.Sequential([  
  keras.Input(shape=(658)),
  layers.Embedding(input_dim=658, output_dim=64),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(12,return_sequences=True, input_shape = (None, 658)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1220, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(dna_code, ytrain, epochs=50, validation_split=0.3, verbose=2)

Epoch 1/50
283/283 - 36s - loss: 6.0490 - accuracy: 0.1160 - val_loss: 3.6384 - val_accuracy: 0.4101
Epoch 2/50
283/283 - 33s - loss: 1.0415 - accuracy: 0.8058 - val_loss: 0.4855 - val_accuracy: 0.9127
Epoch 3/50
283/283 - 33s - loss: 0.1265 - accuracy: 0.9717 - val_loss: 0.3850 - val_accuracy: 0.9357
Epoch 4/50
283/283 - 33s - loss: 0.0643 - accuracy: 0.9851 - val_loss: 0.3748 - val_accuracy: 0.9445
Epoch 5/50
283/283 - 33s - loss: 0.0452 - accuracy: 0.9904 - val_loss: 0.3009 - val_accuracy: 0.9558
Epoch 6/50
283/283 - 33s - loss: 0.0416 - accuracy: 0.9903 - val_loss: 0.3015 - val_accuracy: 0.9540
Epoch 7/50
283/283 - 33s - loss: 0.0273 - accuracy: 0.9931 - val_loss: 0.3117 - val_accuracy: 0.9605
Epoch 8/50
283/283 - 33s - loss: 0.0201 - accuracy: 0.9954 - val_loss: 0.3141 - val_accuracy: 0.9602
Epoch 9/50
283/283 - 33s - loss: 0.0183 - accuracy: 0.9951 - val_loss: 0.3375 - val_accuracy: 0.9571
Epoch 10/50
283/283 - 33s - loss: 0.0781 - accuracy: 0.9775 - val_loss: 0.4219 - val_accura